In [ ]:
import pandas as pd
import numpy as np 

from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict

In [ ]:
df = pd.read_csv('../data/UserItemData.csv') 
df_user = pd.read_csv('../Data/UserData.csv')

In [4]:
import pickle
import joblib
from userClusteringPipeline import ColumnSelector

In [82]:
import importlib
import userClusteringPipeline
importlib.reload(userClusteringPipeline)

<module 'userClusteringPipeline' from 'c:\\Projects\\pes_mtech_group_3\\notebooks\\userClusteringPipeline.py'>

In [13]:
import os
script_dir = os.getcwd()
print(script_dir)

c:\Projects\pes_mtech_group_3\notebooks


In [5]:
user_cluster = joblib.load("../models/KMeans_pipeline.pkl")

In [6]:
user_cluster.predict(df_user[:1])

array([9])

In [9]:
df_user.head()

,Customer_Name,Customer_Type,Gender,Age,guid
0,Stacey Price,Homemaker,F,25,58526ee0-a7c0-480f-a782-2c0c92b3c8ab
1,Michelle Carlson,Professional,F,47,29206584-b0de-4a9f-a228-2ccd7519d732
2,Lisa Graves,Professional,F,34,1200e410-81c6-4604-aff6-9aa124a692a5
3,Mrs. Patricia May,Homemaker,F,44,a1978ec1-8b08-4f03-b9f2-a23317ee6d5e
4,Susan Mitchell,Young Adult,F,22,03a0b807-9cd6-4fff-94a5-25e12548f300


In [10]:
user_cluster.predict(pd.DataFrame(data=[['Homemaker', 'F',36]],columns=['Customer_Type', 'Gender', 'Age']))

array([9])

In [7]:
df_user[:1]

,Customer_Name,Customer_Type,Gender,Age,guid
0,Stacey Price,Homemaker,F,25,58526ee0-a7c0-480f-a782-2c0c92b3c8ab


In [6]:
user_cluster.predict(df_user.loc[[0]])

array([9])

In [7]:
usermap = dict(zip(df_user['guid'],user_cluster.predict(df_user)))

In [8]:
min(usermap.values())

0

In [9]:
max(usermap.values())

15

In [15]:
df['group'] = df['user'].apply(lambda x: usermap[x])

In [17]:
df.isna().sum()

user       0
product    0
rating     0
group      0
dtype: int64

In [18]:
df['group'].value_counts().sort_index()

group
0     1695169
1     1954505
2     1947539
3     1966071
4     1954994
5     1966672
6     1973192
7     1688060
8     1964429
9     1880362
10    1698866
11    1669989
12    1784290
13    1668557
14    1692090
15    1678916
Name: count, dtype: int64

In [ ]:
i = 0
df[df['group']==i].drop([ 'group' ],axis=1)

(1695169, 3)

In [19]:
data = {}
for i in range(max(usermap.values())+1):
    data[i] = df[df['group']==i].drop([ 'group' ],axis=1).reset_index()

In [21]:
for i in range(max(usermap.values())+1):
    print(f'Shape for {i}th data frame is : {data[i].shape}')

Shape for 0th data frame is : (1695169, 4)
Shape for 1th data frame is : (1954505, 4)
Shape for 2th data frame is : (1947539, 4)
Shape for 3th data frame is : (1966071, 4)
Shape for 4th data frame is : (1954994, 4)
Shape for 5th data frame is : (1966672, 4)
Shape for 6th data frame is : (1973192, 4)
Shape for 7th data frame is : (1688060, 4)
Shape for 8th data frame is : (1964429, 4)
Shape for 9th data frame is : (1880362, 4)
Shape for 10th data frame is : (1698866, 4)
Shape for 11th data frame is : (1669989, 4)
Shape for 12th data frame is : (1784290, 4)
Shape for 13th data frame is : (1668557, 4)
Shape for 14th data frame is : (1692090, 4)
Shape for 15th data frame is : (1678916, 4)


In [22]:
reader = Reader(rating_scale=(0, 1))
surprise_data = {}
for i in range(max(usermap.values())+1):
    surprise_data[i] = Dataset.load_from_df(data[i][['user', 'product', 'rating']], reader ) 

In [32]:
for i in range(max(usermap.values())+1):
    trainset, testset = train_test_split(surprise_data[i], test_size=0.2, random_state=42)
    model = SVD()
    model.fit(trainset)
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions)
    joblib.dump(model,'../models/SVD_Group'+str(i)+'.pkl')
    print(f"RMSE on test data for group {i}: {rmse:.4f}\n")

RMSE: 0.2766
RMSE on test data for group 0: 0.2766

RMSE: 0.2643
RMSE on test data for group 1: 0.2643

RMSE: 0.2637
RMSE on test data for group 2: 0.2637

RMSE: 0.2643
RMSE on test data for group 3: 0.2643

RMSE: 0.2642
RMSE on test data for group 4: 0.2642

RMSE: 0.2656
RMSE on test data for group 5: 0.2656

RMSE: 0.2641
RMSE on test data for group 6: 0.2641

RMSE: 0.2758
RMSE on test data for group 7: 0.2758

RMSE: 0.2649
RMSE on test data for group 8: 0.2649

RMSE: 0.2703
RMSE on test data for group 9: 0.2703

RMSE: 0.2788
RMSE on test data for group 10: 0.2788

RMSE: 0.2769
RMSE on test data for group 11: 0.2769

RMSE: 0.2716
RMSE on test data for group 12: 0.2716

RMSE: 0.2770
RMSE on test data for group 13: 0.2770

RMSE: 0.2778
RMSE on test data for group 14: 0.2778

RMSE: 0.2759
RMSE on test data for group 15: 0.2759



In [6]:
df_productset = pd.read_csv('../data/Products.csv')

In [26]:
df_productset.head()

,ProductSet,Product
0,Daily Essentials,Ice Cream
1,Daily Essentials,Vinegar
2,Daily Essentials,Honey
3,Daily Essentials,Bath Towels
4,Daily Essentials,Feminine Hygiene Products


In [7]:
model = joblib.load("../models/SVD_Group0.pkl")

In [8]:
predictions = [model.predict('a7e8f400-138d-4637-b3ff-1e875052dfab', item_id) for item_id in list(df_productset.Product)]

In [26]:
type(predictions[0])

surprise.prediction_algorithms.predictions.Prediction

In [28]:
len(predictions)

81

In [9]:
top_k = 5
top_n = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_k]
print(top_n)

[Prediction(uid='a7e8f400-138d-4637-b3ff-1e875052dfab', iid='Toothpaste', r_ui=None, est=0.11620890622115686, details={'was_impossible': False}), Prediction(uid='a7e8f400-138d-4637-b3ff-1e875052dfab', iid='Eggs', r_ui=None, est=0.09824147720335526, details={'was_impossible': False}), Prediction(uid='a7e8f400-138d-4637-b3ff-1e875052dfab', iid='Coffee', r_ui=None, est=0.09754933144766192, details={'was_impossible': False}), Prediction(uid='a7e8f400-138d-4637-b3ff-1e875052dfab', iid='Milk', r_ui=None, est=0.09327124726880792, details={'was_impossible': False}), Prediction(uid='a7e8f400-138d-4637-b3ff-1e875052dfab', iid='Toilet Paper', r_ui=None, est=0.09056931244719918, details={'was_impossible': False})]


In [ ]:
top_k = 3
top_n = get_top_n(predictions, n=top_k)
true_ratings = build_true_ratings(testset)
precision, recall = precision_recall_at_k(top_n, true_ratings)
print(f"Precision@{top_k}: {precision:.2f}")
print(f"Recall@{top_k}: {recall:.2f}")

In [31]:
top_n

[Prediction(uid='a7e8f400-138d-4637-b3ff-1e875052dfab', iid='Toothpaste', r_ui=None, est=0.12167649541552597, details={'was_impossible': False}),
 Prediction(uid='a7e8f400-138d-4637-b3ff-1e875052dfab', iid='Dish Soap', r_ui=None, est=0.093612569860006, details={'was_impossible': False}),
 Prediction(uid='a7e8f400-138d-4637-b3ff-1e875052dfab', iid='Tea', r_ui=None, est=0.09226491113246549, details={'was_impossible': False})]

In [ ]:
reader = Reader(rating_scale=(0, 1))
#del df, CustomerNames, ProductSet, existing_pairs, unrated_pairs
surprise_data = Dataset.load_from_df(df[['user', 'product', 'rating']], reader)

In [87]:
surprise_data[1]

In [5]:
trainset, testset = train_test_split(surprise_data, test_size=0.2, random_state=42)

In [6]:
testset

[('4369ab2c-7dd0-456d-86d6-cf3177e10b0c', 'Shower Gel', 1.0),
 ('cb13551c-113d-47cf-a6d1-ac06174ca509', 'Vacuum Cleaner', 0.0),
 ('b7834d7a-a461-4243-94be-a5339a6b6e33', 'Butter', 0.0),
 ('191d4427-9467-439c-9ddb-01b368b5abdf', 'Water', 0.0),
 ('40f47a20-6ace-4bff-a0ba-d8891610cffe', 'Eggs', 0.0),
 ('c12c75ef-a8c1-412b-82e6-dd150867a19b', 'Chips', 0.0),
 ('eaa6b110-4dc2-431e-ac5e-14c296994d54', 'Dish Soap', 0.0),
 ('63dabb2d-34c4-4fda-8552-be027c1b453c', 'Mayonnaise', 1.0),
 ('1a9308ad-10df-445f-b041-1ff6506698d3', 'Lawn Mower', 0.0),
 ('9a28f1e8-27fd-4b2e-9d39-0b777a427e09', 'Trash Cans', 0.0),
 ('812c1772-fa8c-430b-938d-ef84d2e92954', 'Milk', 0.0),
 ('4e018f57-abff-4ab7-8a59-93ea87fc1126', 'Cleaning Spray', 0.0),
 ('d4c7516c-5a30-4753-814d-7a8ffeab9638', 'Syrup', 0.0),
 ('bef0e154-ef69-4e38-9180-3bbd9f029517', 'Trash Cans', 0.0),
 ('23f499bc-5767-4946-a6c6-7db1907c64d2', 'Dishware', 0.0),
 ('394221ec-e922-4afe-8710-523a5b5fe3f5', 'Toilet Paper', 0.0),
 ('0940daa6-020e-47b8-86d2-40b8f

In [7]:
model = SVD()
model.fit(trainset)

In [9]:
predictions = model.test(testset)


In [10]:
rmse = accuracy.rmse(predictions)
print(f"\nRMSE on test data: {rmse:.4f}")

RMSE: 0.2665

RMSE on test data: 0.2665


In [89]:
model.test(testset[:5])

[Prediction(uid='07743462-4a2b-4ad4-b01a-bbd3542d15e6', iid='Razors', r_ui=1.0, est=0.2735349760147179, details={'was_impossible': False}),
 Prediction(uid='13a71e79-eb74-4442-8ef4-be718466b88a', iid='Plant Fertilizer', r_ui=0.0, est=0.03177889455306079, details={'was_impossible': False}),
 Prediction(uid='8943f9dd-a2ae-4117-9969-21e2492c91f7', iid='Tomatoes', r_ui=0.0, est=0.042737310395403745, details={'was_impossible': False}),
 Prediction(uid='f52dbb5f-1c95-42ac-ab07-07845931299d', iid='Shrimp', r_ui=0.0, est=0.08955599657267423, details={'was_impossible': False}),
 Prediction(uid='9578822d-ca36-4be0-b0b1-98758bd7ba65', iid='Tuna', r_ui=0.0, est=0, details={'was_impossible': False})]

In [11]:
model.predict(uid='4369ab2c-7dd0-456d-86d6-cf3177e10b0c', iid='Shower Gel')

Prediction(uid='4369ab2c-7dd0-456d-86d6-cf3177e10b0c', iid='Shower Gel', r_ui=None, est=0, details={'was_impossible': False})

In [96]:
model.predict(uid='4369ab2c-7dd0-456d-86d6-cf3177e10b0c', iid='Shower Gel')

Prediction(uid='4369ab2c-7dd0-456d-86d6-cf3177e10b0c', iid='Shower Gel', r_ui=None, est=0.056688784150643066, details={'was_impossible': False})

In [93]:
top_k = 3

In [10]:

# Define helper to get top-N
def get_top_n(predictions, n=5, threshold=0.5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        if est >= threshold:
            top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        top_n[uid] = sorted(user_ratings, key=lambda x: x[1], reverse=True)[:n]
    return top_n

# Build ground truth
def build_true_ratings(testset, threshold=.5):
    truth = defaultdict(set)
    for uid, iid, true_r in testset:
        if true_r >= threshold:
            truth[uid].add(iid)
    return truth

# Calculate Precision@k and Recall@k
def precision_recall_at_k(top_n, true_ratings, k=5):
    precisions = []
    recalls = []
    for uid in top_n:
        pred_items = [iid for (iid, _) in top_n[uid][:k]]
        true_items = true_ratings.get(uid, set())
        if not true_items:
            continue
        num_correct = len(set(pred_items) & true_items)
        precisions.append(num_correct / k)
        recalls.append(num_correct / len(true_items))
    return sum(precisions) / len(precisions), sum(recalls) / len(recalls)

In [14]:
top_k = 3
top_n = get_top_n(predictions, n=top_k)
true_ratings = build_true_ratings(testset)
precision, recall = precision_recall_at_k(top_n, true_ratings)
print(f"Precision@{top_k}: {precision:.2f}")
print(f"Recall@{top_k}: {recall:.2f}")

Precision@3: 0.17
Recall@3: 0.06


In [15]:
top_k = 5
top_n = get_top_n(predictions, n=top_k)
true_ratings = build_true_ratings(testset)
precision, recall = precision_recall_at_k(top_n, true_ratings)
print(f"Precision@{top_k}: {precision:.2f}")
print(f"Recall@{top_k}: {recall:.2f}")

Precision@5: 0.21
Recall@5: 0.08


In [16]:
top_k = 7
top_n = get_top_n(predictions, n=top_k)
true_ratings = build_true_ratings(testset)
precision, recall = precision_recall_at_k(top_n, true_ratings)
print(f"Precision@{top_k}: {precision:.2f}")
print(f"Recall@{top_k}: {recall:.2f}")

Precision@7: 0.21
Recall@7: 0.08
